In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm

# !nbdev_build_lib

In [22]:
inp = pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=500)
df = next(inp)
df = df[['product_name',
        '_keywords' ,
        'ingredients_text',
        'categories',
        'food_groups',
        'nutriments'
        ]]
        

df = df.rename(columns = {'_keywords':'keywords'})
df = df [df['product_name'].str.strip()!=''].dropna(subset = ['product_name'])
df['keywords'] = df['keywords'].apply(lambda x:', '.join(x))


nutriments = pd.json_normalize(df[['nutriments']].to_dict('records')).set_index(df.index)


nutriments.columns = [c.replace('nutriments.','').replace('-','_') for c in nutriments.columns]


df = df.join(nutriments[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']]).drop(columns = ['nutriments'])
df['food_groups'] = df['food_groups'].str.replace('en:','')
df = df.dropna(subset = ['energy_kcal_100g'])


insert_ignore(df,'foods_big',unique_cols=['product_name'])


In [23]:
bigintmax = 2**63-1

In [24]:
for df in tqdm(pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=5000), desc="openfood parsing"):
    df = df[['product_name',
            '_keywords' ,
            'ingredients_text',
            'categories',
            'food_groups',
            'nutriments'
            ]]
            

    df = df.rename(columns = {'_keywords':'keywords'})
    df = df [df['product_name'].str.strip()!=''].dropna(subset = ['product_name'])
    df['keywords'] = df['keywords'].apply(lambda x:', '.join(x))


    nutriments = pd.json_normalize(df[['nutriments']].to_dict('records')).set_index(df.index)


    nutriments.columns = [c.replace('nutriments.','').replace('-','_') for c in nutriments.columns]


    df = df.join(nutriments[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']]).drop(columns = ['nutriments'])
    df['food_groups'] = df['food_groups'].str.replace('en:','')
    df = df.dropna(subset = ['energy_kcal_100g'])


    try: insert_ignore(df,'foods_big',unique_cols=['product_name'])
    except:pass





openfood parsing: 450it [38:39,  5.16s/it]


In [21]:
df

,product_name,keywords,ingredients_text,categories,food_groups,nutriments
id,,,,,,
3.596710e+12,"Petits pois carottes à l'étuvée, extra fins","extra-fin, plante, poi, auchan, base, petit, c...","Petits pois (50 % minimum), carottes (30 % min...","Aliments et boissons à base de végétaux, Alime...",en:vegetables,"{'energy-kj_value': 263, 'carbohydrates_value'..."
3.596710e+12,"Petits pois carottes à l'étuvée, extra fins","fruit, etuvee, fin, vegetaux, vegetale, boisso...","Petits pois (50% minimum*), carottes (30% mini...","Aliments et boissons à base de végétaux, Alime...",en:vegetables,"{'energy_value': 63, 'salt_unit': 'g', 'carboh..."
3.596710e+12,Petits pois carottes à l'étuvée,"auchan, base, petit, conserve, carotte, plante...","Petits pois (50% minimum*), carottes (30% mini...","Aliments et boissons à base de végétaux, Alime...",en:vegetables,"{'carbohydrates_value': 8.8, 'energy-kcal': 63..."
3.596710e+12,"Petits pois carottes à l'étuvée, très fins","legume, en, aliment, petit, conserve, carotte,...","Petits pois (50% minimum*), carottes (30% mini...","Aliments et boissons à base de végétaux, Alime...",en:vegetables,"{'sodium_unit': 'g', 'carbohydrates': 8.8, 'nu..."
3.596710e+12,Petits pois à l'étuvée extra fins,"fin, etuvee, fruit, vegetale, vert, vegetaux, ...",,"Aliments et boissons à base de végétaux, Alime...",en:vegetables,"{'saturated-fat_serving': 0.28, 'saturated-fat..."
...,...,...,...,...,...,...
4.003246e+12,Gemüse und Nudeln mit Huhn,"ohne, milcheiweiß, eu-landwirtschaft, zuckerzu...","Tomatenpüree 23%, Wasser, Karotten 13%, Dinkel...",Babykost ( Ab dem 8. Monat ),NaN,"{'sugars_100g': 2.4, 'fat': 1.9, 'nova-group_s..."
4.003246e+12,Gemüsepolenta mit Kalb,"ab, zusatz, sunsan, ökologische, babykost, von...","Maisgrieß gekocht 47 %, Karotten 14 %, Tomaten...",Babykost ( Ab dem 8. Monat ),NaN,"{'carbohydrates_value': 5.7, 'energy-kj_value'..."
4.003246e+12,Fleischzubereitung Rind,"bio, glutenfrei, zuckerarm, demeter, farbstoff...","Wasser, Rindfleisch 40%, Reisvollkornmehl, Rapsöl",Babykost ( Nach dem 4. Monat ),NaN,"{'energy_100g': 474, 'saturated-fat': 2.1, 'pr..."


In [ ]:
for df in tqdm(pd.read_json(main_path/'openfoodfacts-products.jsonl.gz',lines=True,chunksize=50000), desc="openfood parsing"):